# Neptune + Sacred

## Introduction

This guide will show you how to:

* Initialize Neptune and create a `run`,
* Log Sacred experiment metrics and atrifacts using `NeptuneObserver()`.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install -U neptune-client[sacred] sacred torch torchvision

## Basic Example

### Import Libraries

In [ ]:
import neptune.new as neptune
import torch
import torch.nn as nn
import torch.optim as optim

from neptune.new.integrations.sacred import NeptuneObserver
from sacred import Experiment
from torchvision import datasets, transforms

In [ ]:
if torch.device("cuda:0"):
    torch.cuda.empty_cache()

### Start a run

To connect your script to Neptune and create a new run, we tell Neptune:
* **Who you are** - with a Neptune API token
* **Where to send your data** - to a Neptune project

The cell below lets you record data to the public project [common/sacred-integration](https://app.neptune.ai/o/common/org/sacred-integration) as an anonymous user.

In [ ]:
neptune_run = neptune.init_run(
    project="common/sacred-integration", api_token=neptune.ANONYMOUS_API_TOKEN, tags="notebook"
)

Alternatively, you can log the example to your own workspace.

To do that, replace the code above with the following:

```python
from getpass import getpass

neptune_run = neptune.init_run(
    api_token=getpass("Enter your Neptune API token: "),
    project="workspace-name/project-name",  # replace with your own
)
```

For example, if your workspace name is `ml-team` and the project name is `classification`, the project argument is: `project="ml-team/classification"`.

To find your API token and project name, [log in to Neptune](https://app.neptune.ai/).
- In the top-right corner, click your avatar and select **Get your API token**.
- To find and copy your project name, navigate to the project, then click **Settings** → **Properties**.

### Add NeptuneObserver() to your sacred experiment's observers
Dashboard: [https://app.neptune.ai/o/common/org/sacred-integration/e/SAC-11/dashboard/Sacred-Dashboard-6741ab33-825c-4b25-8ebb-bb95c11ca3f4](https://app.neptune.ai/o/common/org/sacred-integration/e/SAC-11/dashboard/Sacred-Dashboard-6741ab33-825c-4b25-8ebb-bb95c11ca3f4)

By using NeptuneObserver(), the following is automatically logged to the Neptune app for you:
- Hyperparameters
- Loss
- Metrics


In [ ]:
ex = Experiment("image_classification", interactive=True)
ex.observers.append(NeptuneObserver(run=neptune_run))

Define Model

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, input_sz=32 * 32 * 3, n_classes=10):
        super(BaseModel, self).__init__()
        self.lin = nn.Linear(input_sz, n_classes)

    def forward(self, input):
        x = input.view(-1, 32 * 32 * 3)
        return self.lin(x)

In [ ]:
model = BaseModel()

Define your configuration/hyperparamenters

In [ ]:
@ex.config
def cfg():
    data_dir = "data/CIFAR10"
    data_tfms = {
        "train": transforms.Compose(
            [
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )
    }
    lr = 1e-2
    bs = 128
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Define your run

In [ ]:
@ex.main
def run(data_dir, data_tfms, lr, bs, device, _run):
    trainset = datasets.CIFAR10(data_dir, transform=data_tfms["train"], download=True)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=bs, shuffle=True, num_workers=2)
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    for i, (x, y) in enumerate(trainloader, 0):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model.forward(x)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, y)
        acc = (torch.sum(preds == y.data)) / len(x)

        # Log loss
        ex.log_scalar("training/batch/loss", loss)
        # Log accuracy
        ex.log_scalar("training/batch/acc", acc)

        loss.backward()
        optimizer.step()

    return {"final_loss": loss.item(), "final_acc": acc.cpu().item()}

### Run your experiment and explore metadata in the Neptune app

All metadata is logged automatically to Neptune. 

After running your script or notebook cell you will get a link similar to:
https://app.neptune.ai/o/common/org/sacred-integration/e/SAC-1
with common/sacred-integration replaced by your project, and SAC-1 replaced by your run.

Click on the link to open the run in Neptune and watch your model training live.

In [ ]:
ex.run()

## More Options

### Log Artifacts

Model architecture and weights

In [ ]:
model_fname = "model"
print(f"Saving model archictecture as {model_fname}.txt")
with open(f"{model_fname}_arch.txt", "w") as f:
    f.write(str(model))

In [ ]:
print(f"Saving model weights as {model_fname}.pth")
torch.save(model.state_dict(), f"./{model_fname}.pth")

In [ ]:
ex.add_artifact(filename=f"./{model_fname}_arch.txt", name=f"{model_fname}_arch")
ex.add_artifact(filename=f"./{model_fname}.pth", name=model_fname)

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
neptune_run.stop()